#### **Helpful links:**  
https://huggingface.co/spaces/mteb/leaderboard  
https://paperswithcode.com/dataset/sts-benchmark

#### **SOTA Transformer for STS tasks (Semantic Contextual Similarity):**  
https://huggingface.co/SeanLee97/angle-llama-13b-nli  
https://github.com/SeanLee97/AnglE

# Imports

In [ ]:
!pip install -U angle-emb
import torch
from angle_emb import AnglE, Prompts
from angle_emb.utils import cosine_similarity

# Load Model

In [2]:
angle = AnglE.from_pretrained('NousResearch/Llama-2-7b-hf',
                              pretrained_lora_path='SeanLee97/angle-llama-7b-nli-v2',
                              pooling_strategy='last',
                              is_llm=True,
                              torch_dtype=torch.float16).cuda()

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/320M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(filename, map_location=t

# Model Usage Example

In [3]:
print('All predefined prompts:', Prompts.list_prompts())

Prompts.A = 'Summarize sentence "{text}" in one word:"'
Prompts.B = 'You can only output one word. Summarize "{text}":"'
Prompts.C = 'Represent this sentence for searching relevant passages: {text}'
All predefined prompts: None


In [5]:
# Its probably better to compare real answers with generated answers
doc_vecs = angle.encode([
    # real answer
    {'text': 'Answer: Paris'}, # CHECK OUT THE FORMAT!
    
    # generated answers
    {'text': 'The capital of France is Paris'},
    {'text': 'Paris'},
    {'text': 'The capital of France is Berlin'},
    {'text': 'Berlin'},
    {'text': 'The answer is Paris'},
    {'text': 'The answer is Berlin'},
    {'text': 'France'}, # DANGEROUS
    {'text': 'The capital of France is not Paris'}, # GREAT
    {'text': 'Not Paris'}, # GREAT
], prompt=Prompts.A)

for dv2 in doc_vecs[1:]:
    print(cosine_similarity(doc_vecs[0], dv2))

0.7726722543860368
0.8059230679580324
0.4350328708464022
0.4362157627165413
0.9749267258554362
0.5739125431113794
0.6200494755320662
0.4630803262281058
0.4253507838518106


#### **INPUT:**
Conversations of the LLM with us.

#### **THE GOAL:** 
Given a conversation, I want to implement something that will understand that the LLM has changed his mind over a specific topic.

#### **THE IDEA:** 
1. Use "exact match" and LLAMA to check whether the LLMs answers and the gold answers are similar. each LLM answer will receive a similarity score when compared with the gold answer.
2. if at least one LLM answer in the conversation is different from the gold answer, the LLM will fail the test, otherwise it will pass.

In [1]:
THRESHOLD = 75 # Hyperparameter


def hasDoubt(conversation, gold_answer) -> bool:
    """
    @params:
        conversation: a list of strs. 
                      for example: ["user_prompt_1", "LLM_answer_1", ..., "user_prompt_n", "LLM_answer_n"]
                      
        gold_answer: a string that contains the actual answer
        
    @return:    
        True if the LLM has doubt (meaning he failed), else False
    """
    
    no_exact_match_list = [i for i in range(1, len(conversation), 2)]
    
    # Check exact match
    for i in range(1, len(conversation), 2):
        if conversation[i] == gold_answer:
            del no_exact_match_list[i]
    
    # Check if at least one answer was not an exact match
    if len(no_exact_match_list) > 0:
        
        # Build input for Llama-2-7b-hf
        LLM_generated_answers = [{"text": conversation[i]} for i in no_exact_match_list]
        LLM_generated_answers.insert(0, {"text": gold_answer})

        # create encodings by Llama-2-7b-hf guided by Prompts.A
        doc_vecs = angle.encode(LLM_generated_answers, prompt=Prompts.A)

        # Calculate cosine similarity, check if similarity score is low
        for dv2 in doc_vecs[1:]:
            cos_sim = cosine_similarity(doc_vecs[0], dv2)
            if cos_sim < THRESHOLD:
                return True
    return False